In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import json
import os
if '../src' not in sys.path:
    sys.path.append('../src')
import utils
import settings
from retriever import Retriever
from chatbot import Chatbot
from IPython.display import Markdown, display
from langchain_openai import ChatOpenAI

In [3]:
logger = utils.get_logger("03_rag")
wrapper = utils.get_text_wrapper()

### Document retriever

The document retriever is responsible for finding relevant documents based on the user question. It initializes an Opensearch client, uses a `neural` query for semantic search.

Use the `MODEL_ID` of the embedding model used at ingestion time to generate vector embeddings for the queries' text.

[OpenSearch Python Client docs](https://opensearch-project.github.io/opensearch-py/api-ref/clients/opensearch_client.html)

In [4]:
retriever = Retriever(logger=logger, index=settings.INDEX_NAME)

In [5]:
query_text = "Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?"

docs = retriever.retrieve_documents(query_text=query_text)

retriever.print_documents(docs)

{
    "page_content":"| Pentru candidații care au optat la programe de studii școlarizate în limbi străine (engleză, franceză, germană), dacă este cazul,
documentul/documentele care fac dovada cunoașterii limbii respective. Originalul documentelor din Tabelul Anx.VII-2— Punctul 10. |\n|:--|\n| Pentru candidații
care au optat la domeniul Ştiințe ale educației, eseul motivațional al candidatului relativ la capacitatea, dorința și determinarea acestuia de a urma studiile
în cadrul domeniului Ştiințe ale educației. Pentru candidații care optează la domeniul Teologie, Binecuvântarea Înaltpreasfințitului Părinte Arhiepiscop sau,
după caz, Episcop, ierarhul eparhiei din care provine candidatul (de unde are domiciliul), în vederea înscrierii la Facultatea de Teologie, Litere, Istorie și
Arte. Pentru candidații care au optat la domeniul Inginerie industrială — învățământ dual eseul motivațional al candidatului relativ la capacitatea, dorința și
determinarea acestuia de a urma studiile universit

In [6]:
# display(Markdown(docs[1].page_content))

### RAG pipeline

Initialize llm

In [7]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

llm = ChatOpenAI(model=settings.OPENAI_MODEL, temperature=0, top_p=1)

In [8]:
chatbot = Chatbot(retriever=retriever, llm=llm)

In [9]:
# chatbot.delete_conversation_history()

In [10]:
chatbot.run("Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?")
print(chatbot.refined_question)
# print(chatbot.formatted_prompt)
chatbot._print_response()

Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?
UTILIZATOR:
Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru a demonstra cunoștințele lingvistice la înscrierea la un program de studii în limba engleză, candidații pot prezenta următoarele documente:

1. **Liceu
absolvit în respectiva limbă de predare**: Dacă acest lucru nu este evident din celelalte documente furnizate la înscriere, candidatul trebuie să încarce o
copie a unui document eliberat de liceu care să ateste că a urmat un program de studii în limba engleză.
   
2. **Certificat de competențe lingvistice**: Un
certificat care atestă competențele lingvistice în limba engleză, cu un nivel minim B2.

3.

In [11]:
chatbot.run("Cand are loc testul online de limba de la politehnica?")
print(f"Refined question: {chatbot.refined_question}")
# print(chatbot.formatted_prompt)
chatbot._print_response()

Refined question: Când se va desfășura testul online de limbă organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cand are loc testul online de limba de la politehnica?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Testul online de limbă organizat de Universitatea Politehnica din București se va desfășura în două sesiuni. Prima sesiune va avea loc pe data de 21 martie
2025, dată valabilă pentru toți candidații. A doua sesiune este programată pentru 23 mai 2025, dar această dată este valabilă doar pentru candidații care s-au
înscris la facultățile unde admiterea anticipată se face pe baza rezultatelor la activitățile școlare (dosar) (c-s, c-v, c-m) (sursa:
https://www.upit.ro/_document/305986/regulament_admitere_licenta_2025.pdf, pagina 26).

Surse:
-
https://www.upit.ro/_document/305986/regulament_admitere_licenta_2025.pdf: pagina 26
-
https://www.u

In [12]:
# chatbot.run("Cum ma inscriu la testul online de limba de la politehnica?")
# chatbot._print_response()

In [13]:
chatbot.run("Cum ma inscriu la test?")
print(f"Refined question: {chatbot.refined_question}")
chatbot._print_response()

Refined question: Cum pot să mă înscriu la testul online de limbă organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cum ma inscriu la test?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru a vă înscrie la testul online de limbă organizat de Universitatea Politehnica din București, trebuie să urmați următorii pași:

1. **Înscriere**:
Completați formularul de înscriere disponibil la adresa https://forms.office.com/e/xrMTzESmi7. Este important ca adresa de e-mail completată în formular să fie
aceeași cu cea utilizată în cadrul procesului de înscriere la concursul de admitere de pe pagina http://admitere.pub.ro/. Formularul trebuie completat până cel
târziu pe 19 martie 2025, ora 12:00. Toate câmpurile din formular sunt obligatorii, iar candidații care nu completează formularul la timp și în integralitate nu
vor putea susține testul.

2. **Obținer

In [14]:
# chatbot.delete_last_message()

In [15]:
# chatbot.state["conversation_history"]

In [16]:
docs = chatbot.state["documents"]
for doc in docs:
    print(json.dumps(doc.metadata, indent=4, ensure_ascii=False))

{
    "score": 0.12076896,
    "filename": "regulament_admitere_licenta_2025.pdf",
    "page_number": 26,
    "id": "17e6c0b294562ed730f2af76af55a3331d661ac200ff7b50f0ff0c685e53630d-36",
    "text": "Candidații care la înscriere optează pentru un domeniu cu limbă de predare engleză, franceză sau germană și care aleg să probeze cunoașterea respectivei limbi prin promovarea testului POLITEHNICA București, vor susține acest test on-line, la data de 21 martie 2025 (dată valabilă pentru toți candidații) sau 23 mai 2025 (această a doua dată este valabilă doar pentru candidații înscriși la facultățile la care admiterea anticipată este pe baza rezultatelor la activitățile școlare (dosar)). Detalii privind organizarea sa pot fi găsite la https://admitere.pub.ro. Ses. I-4.9. La înscriere, în formularul tipizat menționat anterior, în ordinea preferințelor sale, candidatul poate face opțiuni privind domeniile (numai de la facultatea la care s-a înscris) la care acesta dorește să fie pre-admis. În 

In [17]:
docs = retriever.retrieve_documents(query_text="Cum pot să mă înscriu la testul online de limbă organizat de Universitatea Politehnica din București?")


In [18]:
retriever.print_documents(docs)


{
    "page_content":"Candidații care la înscriere optează pentru un domeniu cu limbă de predare engleză, franceză sau germană și care aleg să probeze
cunoașterea respectivei limbi prin promovarea testului POLITEHNICA București, vor susține acest test on-line, la data de 21 martie 2025 (dată valabilă pentru
toți candidații) sau 23 mai 2025 (această a doua dată este valabilă doar pentru candidații înscriși la facultățile la care admiterea anticipată este pe baza
rezultatelor la activitățile școlare (dosar)). Detalii privind organizarea sa pot fi găsite la https://admitere.pub.ro. Ses. I-4.9. La înscriere, în formularul
tipizat menționat anterior, în ordinea preferințelor sale, candidatul poate face opțiuni privind domeniile (numai de la facultatea la care s-a înscris) la care
acesta dorește să fie pre-admis. În Sesiunea I sunt disponibile numai locuri cu finanțare de la buget. Ses. I-4.10. Fiecărui candidat cu dosar de înscriere
validat de comisia locală i se întocmește o legitimație de

In [19]:
print(chatbot.formatted_prompt)


            Ești un asistent care răspunde la întrebări pe baza contextului din documentele relevante.
            Generează un răspuns formal pe baza datelor primite. Pentru fiecare afirmație furnizează explicit sursa extrasă din metadatele documentului.
            Dacă nu poți formula un răspuns pe baza datelor primite, spune că nu știi, nu încerca să inventezi un răspuns.

            Documente relevante:
            Sursa: https://www.upit.ro/_document/305986/regulament_admitere_licenta_2025.pdf, pagina: 26
Conținut: Candidații care la înscriere optează pentru un domeniu cu limbă de predare engleză, franceză sau germană și care aleg să probeze cunoașterea respectivei limbi prin promovarea testului POLITEHNICA București, vor susține acest test on-line, la data de 21 martie 2025 (dată valabilă pentru toți candidații) sau 23 mai 2025 (această a doua dată este valabilă doar pentru candidații înscriși la facultățile la care admiterea anticipată este pe baza rezultatelor la activitățil